In [1]:
import tensorflow as tf

sess = tf.InteractiveSession()

In [2]:
# 画像をグレースケールで読み込み、浮動小数点数データに変換する。
# tf.read_file()はファイルシステムからファイルの内容(バイト列)を取得する。
png = tf.read_file('03-02-original.png')
# バイト列をテンソルに変換(デコード)する。
image = tf.image.decode_png(png, channels=1, dtype=tf.uint8)  
image_float = tf.to_float(image)

Instructions for updating:
Use `tf.cast` instead.


In [3]:
# tf.nn.conv2dメソッドを適用するために4階のテンソルに変換
# 4階のテンソルはそれぞれ「バッチサイズ、画像横幅、画像縦幅、チャンネル」
image_reshape = tf.reshape(image_float, [-1, 32, 32, 1])

In [4]:
# カーネル(フィルター)の作成
kernel = tf.constant(
    [
        [0, -1, -1, -1, 0],
        [-1, 0, 3, 0, -1],
        [-1, 3, 0, 3, -1],
        [-1, 0, 3, 0, -1],
        [0, -1, -1, -1, 0]
    ],
    dtype=tf.float32)

In [5]:
# tf.nn.conv2dメソッドを適用するために4階のテンソルに変換
# 4階のテンソルはそれぞれ「横幅、縦幅、入力チャンネル、出力チャンネル」
kernel_reshape = tf.reshape(kernel, [5, 5, 1, 1])

In [6]:
# ストライド幅3
# 3ピクセルずつ動かす
strides = [1, 3, 3, 1]

In [7]:
# 畳み込み tf.nn.conv2d()
convolution_result = tf.nn.conv2d(
    image_reshape,
    kernel_reshape,
    strides=strides,
    # パディング：'VALID'はパディング無効
    padding='VALID'   
)

In [8]:
# 結果を見やすいように2階のテンソル(行列)に変換
tf.reshape(convolution_result, [10, 10]).eval()

array([[  28., -165.,   84.,  -76.,  -77.,  193.,   48.,  113.,  -75.,
         -71.],
       [   8.,   60.,   -2.,   30.,  -89.,  -17.,  119.,  -75.,  -37.,
          27.],
       [  72., -221., -123., -147.,  -28.,  125., -104.,    2.,   20.,
         -59.],
       [ 133.,  109.,  206.,  173.,   97.,   92.,    1.,  -56.,  -22.,
          54.],
       [-286.,   47.,  -45.,  103.,  -51.,  -99.,   13.,  110.,  -22.,
          78.],
       [ 112.,   49.,   64.,  -91.,  -29., -180.,   87.,   -7.,  210.,
         133.],
       [ -49.,  -88.,   53.,  -77.,  -42.,  110., -103.,  116., -280.,
          88.],
       [ -72.,    3.,   79.,  176., -194.,  139., -319.,   57.,  175.,
        -177.],
       [ 161., -136.,  -51.,  107.,    1.,  -10.,  560., -105.,   93.,
         -99.],
       [ -64.,   44.,   85.,   52.,  -59.,   71., -101.,   -1.,  -98.,
          81.]], dtype=float32)

In [9]:
# 畳み込み
convolution_result_with_padding = tf.nn.conv2d(
    image_reshape,
    kernel_reshape,
    strides=strides,
    # パディング：'SAME'はゼロパディング(画像の周囲に数値0のパディングを埋める)
    padding='SAME'   
)

In [10]:
# 結果を見やすいように2階のテンソル(行列)に変換
tf.reshape(convolution_result_with_padding, [11, 11]).eval()

array([[  76.,  238.,  221.,  321.,  194.,  326.,  346.,  115.,  166.,
          99.,  386.],
       [ 196.,  -16.,   53.,   27.,   32.,   62.,   53., -168.,  -17.,
         174.,   65.],
       [ 254.,  -57., -115.,   -9.,   59.,    6.,   88.,  101.,  -45.,
          25.,  140.],
       [ 301.,  -68., -122.,   70.,  161., -190.,    4.,   74., -174.,
        -109.,  233.],
       [  94.,   89.,    6.,   23.,   91., -119., -131.,  -57.,  328.,
          41.,  119.],
       [ 117.,   76.,   38., -165., -163.,  141.,   90.,   44.,   29.,
          53.,   30.],
       [ 236.,  -44.,   79.,   53.,    1.,   49.,  -46.,  145., -101.,
         -76.,   59.],
       [ 408., -203.,    8.,   85.,  -46.,   43.,  -26., -259.,  109.,
         -97.,   97.],
       [ 136.,  143.,  -26., -101.,   35.,   -5.,  359.,  339., -167.,
        -127.,   89.],
       [ 310.,  -71.,  -69.,  110., -184.,  -75., -201.,    9.,  -74.,
           4.,  282.],
       [ 327.,  259.,  149.,   80.,  167.,  212.,  161.,  -2

In [11]:
# 最大値プーリング tf.nn.max_pool()
pooling_result = tf.nn.max_pool(
    convolution_result,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='VALID'
)

In [12]:
# 出力用のサイズ調整
tf.reshape(pooling_result, [5, 5]).eval()

array([[ 60.,  84., 193., 119.,  27.],
       [133., 206., 125.,   2.,  54.],
       [112., 103., -29., 110., 210.],
       [  3., 176., 139., 116., 175.],
       [161., 107.,  71., 560.,  93.]], dtype=float32)

In [13]:
# 仮想のニューラルネットワークの出力
y = tf.constant([1.1, 3.2, -0.9])

In [14]:
# ソフトマックス関数 tf.nn.softmax()
p = tf.nn.softmax(y)
p.eval()

array([0.10750949, 0.87794065, 0.01454983], dtype=float32)